In [2]:
import xarray as xr
import pandas as pd
import plotly.express as px

## Import data


In [1]:
station = "Calcofi"

In [3]:
obs_data = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr")
obs_data

<xarray.Dataset> Size: 102kB
Dimensions:    (time: 156, latitude: 9, longitude: 9, layer: 1)
Coordinates:
  * latitude   (latitude) float64 72B 29.5 30.5 31.5 32.5 ... 35.5 36.5 37.5
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 72B -125.5 -124.5 -123.5 ... -118.5 -117.5
  * time       (time) datetime64[ns] 1kB 2002-01-31 2002-02-28 ... 2014-12-31
Data variables:
    day        (time, latitude, longitude, layer) float32 51kB ...
    night      (time, latitude, longitude, layer) float32 51kB ...
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [4]:
cafe_data = xr.open_dataset(f"../data/1_products/{station}_cafe.zarr", engine="zarr")
cafe_data = cafe_data.resample(time="1D").mean(dim="time").interpolate_na(dim="time", method="linear").CAFE
first_year = str(obs_data.indexes["time"].to_pydatetime()[0].year)
last_year = str(obs_data.indexes["time"].to_pydatetime()[-1].year)
cafe_data = cafe_data.sel(time=slice(first_year, last_year)).sel(
    latitude=obs_data.latitude, longitude=obs_data.longitude
)
cafe_data

<xarray.DataArray 'CAFE' (time: 4748, latitude: 9, longitude: 9)> Size: 3MB
array([[[187.70641327, 210.44682312, 177.77169037, ..., 245.20661545,
         345.16986084, 346.02754974],
        [200.96617508, 184.87548447, 204.69929123, ..., 279.9656105 ,
         359.36453247, 308.23486328],
        [260.63409042, 224.87198257, 201.56934738, ..., 364.25556946,
         449.99412537, 364.95314789],
        ...,
        [311.57821655, 344.64048767, 375.8311615 , ...,          nan,
                  nan,          nan],
        [345.84145355, 372.57913971, 415.71337128, ...,          nan,
                  nan,          nan],
        [278.20756149, 376.42963409, 355.39595032, ...,          nan,
                  nan,          nan]],

       [[187.9651022 , 206.19159079, 178.64610958, ..., 240.17440224,
         333.48835945, 341.91321659],
        [199.11328316, 185.14120436, 206.85678244, ..., 288.37373972,
         353.40550804, 304.95729065],
        [257.54169989, 227.22356319, 201.49915123, ..., 372.95083904,
         447.40411282, 366.08804989],
...
        [386.97064209, 421.74104614, 405.93526154, ...,          nan,
                  nan,          nan],
        [370.92896576, 463.16829529, 513.46253815, ...,          nan,
                  nan,          nan],
        [353.93153839, 424.4833252 , 459.33255615, ...,          nan,
                  nan,          nan]],

       [[165.09898224, 205.35970306, 197.18847885, ..., 248.50347595,
         267.57532959, 232.64947968],
        [194.95214386, 226.23121719, 209.57670822, ..., 254.73242722,
         274.5449173 , 301.25087891],
        [198.24517136, 182.71941757, 199.38319626, ..., 265.73787918,
         317.83043518, 346.83701782],
        ...,
        [366.91705322, 411.35204468, 395.18079224, ...,          nan,
                  nan,          nan],
        [354.35804901, 457.70348663, 500.84984131, ...,          nan,
                  nan,          nan],
        [341.37064209, 414.26619873, 445.9627182 , ...,          nan,
                  nan,          nan]]])
Coordinates:
  * latitude   (latitude) float64 72B 29.5 30.5 31.5 32.5 ... 35.5 36.5 37.5
  * longitude  (longitude) float64 72B -125.5 -124.5 -123.5 ... -118.5 -117.5
  * time       (time) datetime64[ns] 38kB 2002-01-01 2002-01-02 ... 2014-12-31
Attributes: (12/13)
    adg wavelength:               443nm
    aph wavelength:               443nm
    bbp wavelength:               443nm
    doi:                          10.1002/2016GB005521
    long_name:                    primary_production_using_CAFE_model
    mld_criteria:                 0.125 kg m^-3
    ...                           ...
    reference:                    Silsbe_et_al_2016
    remote_sensing_data_product:  OCCCIv6
    resolution:                   0.00f; // float
    standard_name:                NPP
    units:                        mg m-2 day-1
    weight:                       carbon

In [25]:
px.scatter_map(
    cafe_data.isnull().sum(dim="time").to_dataframe().reset_index(),
    lat="latitude",
    lon="longitude",
    color="CAFE",
).update_traces(marker={"size": 20})

## Create climatology


In [26]:
cafe_data_climato = cafe_data.groupby("time.dayofyear").mean()

all_years = []
for year in range(int(cafe_data.time.min().dt.year), int(cafe_data.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = cafe_data_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
cafe_data_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
cafe_data_climato_expanded.time.attrs = cafe_data.time.attrs
cafe_data_climato_expanded.attrs["description"] = "CAFE climatology (2002-2015) interpolated to daily frequency"
cafe_data_climato_expanded

<xarray.DataArray 'CAFE' (time: 4748, latitude: 9, longitude: 9)> Size: 3MB
array([[[199.46180637, 205.72476578, 228.82702607, ..., 300.86054406,
         324.4936318 , 348.38014984],
        [222.15694075, 233.50440216, 246.13822262, ..., 334.93879406,
         390.89109098, 372.00191087],
        [240.88682175, 249.71666248, 276.01794463, ..., 383.12420713,
         416.56164023, 426.32680159],
        ...,
        [349.82029519, 357.85932922, 389.83510854, ...,          nan,
                  nan,          nan],
        [356.05778063, 378.91266427, 414.7987307 , ...,          nan,
                  nan,          nan],
        [346.0818769 , 373.81049347, 402.86431738, ...,          nan,
                  nan,          nan]],

       [[199.35966759, 205.49473509, 227.72284279, ..., 299.18675767,
         322.68462874, 346.94182323],
        [220.70921469, 232.6709985 , 248.12140821, ..., 335.78768121,
         388.89281493, 371.59196186],
        [242.50641981, 253.67160933, 276.8937876 , ..., 385.5068476 ,
         417.83681892, 425.32258591],
...
        [361.21750588, 368.34931649, 394.76917243, ...,          nan,
                  nan,          nan],
        [355.13577978, 377.93423051, 404.55028422, ...,          nan,
                  nan,          nan],
        [335.07091842, 365.74552102, 389.65574341, ...,          nan,
                  nan,          nan]],

       [[199.34270741, 209.66948119, 229.35309323, ..., 304.52149703,
         321.91740069, 343.97966377],
        [225.01290017, 238.70537495, 247.33907793, ..., 334.87926362,
         384.26925299, 376.14305788],
        [239.40133317, 251.71978995, 280.2794555 , ..., 381.37734682,
         416.94917098, 435.24965492],
        ...,
        [357.65708364, 365.34096191, 393.09267962, ...,          nan,
                  nan,          nan],
        [356.31965622, 381.27934979, 412.01606915, ...,          nan,
                  nan,          nan],
        [342.3345897 , 370.91681053, 398.19542972, ...,          nan,
                  nan,          nan]]])
Coordinates:
  * latitude   (latitude) float64 72B 29.5 30.5 31.5 32.5 ... 35.5 36.5 37.5
  * longitude  (longitude) float64 72B -125.5 -124.5 -123.5 ... -118.5 -117.5
  * time       (time) datetime64[ns] 38kB 2002-01-01 2002-01-02 ... 2014-12-31
Attributes: (12/14)
    adg wavelength:               443nm
    aph wavelength:               443nm
    bbp wavelength:               443nm
    doi:                          10.1002/2016GB005521
    long_name:                    primary_production_using_CAFE_model
    mld_criteria:                 0.125 kg m^-3
    ...                           ...
    remote_sensing_data_product:  OCCCIv6
    resolution:                   0.00f; // float
    standard_name:                NPP
    units:                        mg m-2 day-1
    weight:                       carbon
    description:                  CAFE climatology (2002-2015) interpolated t...

## Plot


In [27]:
table_cafe = (
    pd.DataFrame(
        {
            "CAFE": cafe_data.mean(["latitude", "longitude"]).to_dataframe()["CAFE"],
            "CAFE_climato": cafe_data_climato_expanded.mean(["latitude", "longitude"]).to_dataframe()["CAFE"],
        }
    )
    .stack()
    .reset_index()
    .rename(columns={"level_1": "type", 0: "NPP"})
)

In [28]:
px.line(table_cafe, x="time", y="NPP", color="type", title=f"CAFE vs Climato : {station}")

## Export


In [29]:
cafe_data_climato_expanded.to_zarr(f"../data/1_products/{station}_cafe_climato.zarr", mode="w")